# Alabama - Analysis

## Import the Required Packages

In [ ]:
# For working with graphs
import networkx as nx
# For making plots
import matplotlib.pyplot as plt
import numpy as np

# Needed for gerrychain
import gerrychain   
from gerrychain import Graph, Partition, proposals, updaters, constraints, accept, MarkovChain, GeographicPartition
from gerrychain.updaters import cut_edges, Tally
from gerrychain.tree import recursive_tree_part
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from functools import partial
import geopandas as gpd
import pandas as pd

# Other necessary packages
import random
import maup
maup.progress.enabled = True

## Initial Plan - Struck down by Courts

In [2]:
# Read shapefile data into a geo data frame and a graph
al_init_gdf = gpd.read_file("al_cong_201/2021 Alabama Congressional Plan_shape file.shp")
al_init_graph = Graph.from_file("/Users/elenamiller/Desktop/Fall 2024/Math195/Coding/Math195-Final-Project/al_cong_201/2021 Alabama Congressional Plan_shape file.shp", ignore_errors=True)

/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/gerrychain/graph/graph.py:266: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()
/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column LOCKED!
  warnings.warn("NA values found in column {}!".format(column))
/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column NAME!
  warnings.warn("NA values found in column {}!".format(column))


In [3]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
al_init_gdf

,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,WHITE,BLACK,F18_POP,F18_WHT,F18_BLK,F18_IND,F18_ASN,F18_HWN,F18_OTH,IDEAL_VALU,DEVIATION,F_DEVIATIO,F_WHITE,F_BLACK,F_18_POP,F_18_WHT,F_18_BLK,F_18_IND,F_18_ASN,F_18_HWN,F_18_OTH,DISTRICT_L,geometry
0,2,6612.89502,1,1.0,None,CD1,717754,461324,186921,557535,371902,138128,6381,8395,290,7947,717754.0,0.0,0.000000,0.642733,0.260425,0.776777,0.667047,0.247748,0.011445,0.015057,0.000520,0.014254,1|0%,"POLYGON ((-87.59883 30.99745, -87.59884 30.997..."
1,3,10532.34270,2,1.0,None,CD2,717755,433244,217392,557677,350279,162714,2628,10399,307,9802,717754.0,1.0,0.000001,0.603610,0.302878,0.776974,0.628104,0.291771,0.004712,0.018647,0.000550,0.017576,2|0%,"POLYGON ((-85.91929 32.27438, -85.91237 32.274..."
2,4,8460.39062,3,1.0,None,CD3,717754,479432,176953,564281,386048,136382,2048,9869,340,8505,717754.0,0.0,0.000000,0.667961,0.246537,0.786176,0.684141,0.241692,0.003629,0.017490,0.000603,0.015072,3|0%,"POLYGON ((-86.55384 33.01893, -86.55355 33.018..."
3,5,9058.35644,4,1.0,None,CD4,717754,582698,51929,556133,463433,39834,5475,3427,245,18651,717754.0,0.0,0.000000,0.811835,0.072349,0.774824,0.833313,0.071627,0.009845,0.006162,0.000441,0.033537,4|0%,"POLYGON ((-86.14981 34.53363, -86.14864 34.533..."
4,6,3502.37963,5,1.0,None,CD5,717754,499707,124642,561187,403155,95757,4130,10814,447,15080,717754.0,0.0,0.000000,0.696209,0.173656,0.781865,0.718397,0.170633,0.007359,0.019270,0.000797,0.026872,5|0%,"MULTIPOLYGON (((-86.14981 34.53363, -86.14981 ..."
5,7,3261.12426,6,1.0,None,CD6,717754,498843,138019,552286,397498,100878,2183,10568,254,16611,717754.0,0.0,0.000000,0.695006,0.192293,0.769464,0.719732,0.182655,0.003953,0.019135,0.000460,0.030077,6|0%,"POLYGON ((-87.18652 33.55695, -87.18712 33.557..."
6,8,11021.27630,7,1.0,None,None,717754,265204,400306,568067,222731,308030,1707,7036,232,10629,717754.0,0.0,0.000000,0.369491,0.557720,0.791451,0.392086,0.542242,0.003005,0.012386,0.000408,0.018711,7|0%,"POLYGON ((-87.26692 33.51293, -87.26718 33.513..."


We need to confirm that there aren't any issues in the Districts, so we can use the "maup" package.

In [4]:
# Check geodataframe using maup's doctor function
maup.doctor(al_init_gdf)

/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/maup/repair.py:331: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlaps = inters[inters.area > 0].make_valid()


True

## The Second Plan

In [5]:
# Read shapefile data into a geo data frame and a graph
al_adopt_gdf = gpd.read_file("/Users/elenamiller/Desktop/Fall 2024/Math195/Coding/Math195-Final-Project/al_cong_adopted_2023/SP_Remedial_Plan_3 2023-10-05.shp")
al_adopt_graph = Graph.from_file("/Users/elenamiller/Desktop/Fall 2024/Math195/Coding/Math195-Final-Project/al_cong_adopted_2023/SP_Remedial_Plan_3 2023-10-05.shp", ignore_errors=True)

/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/gerrychain/graph/graph.py:266: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()
/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column LOCKED!
  warnings.warn("NA values found in column {}!".format(column))
/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/gerrychain/graph/graph.py:457: UserWarning: NA values found in column NAME!
  warnings.warn("NA values found in column {}!".format(column))


In [6]:
al_adopt_gdf

,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,WHITE,BLACK,AP_WHT,AP_BLK,F18_POP,F18_WHT,F18_BLK,F18_AP_WHT,F18_AP_BLK,IDEAL_VALU,DEVIATION,F_DEVIATIO,F_WHITE,F_BLACK,F_AP_WHT,F_AP_BLK,F_18_POP,F_18_WHT,F_18_BLK,F_18_AP_WH,F_18_AP_BL,DISTRICT_L,geometry
0,1,8126.240234,1,1.0,None,None,717754,527330,116462,566761,126801,557336,421681,86271,446497,90582,717754.0,0.0,0.000000,0.734695,0.162259,0.789631,0.176664,0.776500,0.756601,0.154792,0.801127,0.162527,1|0%,"POLYGON ((-88.19693 31.14404, -88.19317 31.144..."
1,2,9562.906250,2,1.0,None,None,717754,303461,353228,328547,364985,558661,251320,265638,267394,272023,717754.0,0.0,0.000000,0.422792,0.492130,0.457743,0.508510,0.778346,0.449861,0.475491,0.478634,0.486920,2|0%,"POLYGON ((-85.88615 32.49305, -85.88575 32.493..."
2,3,6873.106934,3,1.0,None,None,717754,508080,146376,539477,156204,564595,408820,112550,429007,116843,717754.0,0.0,0.000000,0.707875,0.203936,0.751618,0.217629,0.786614,0.724094,0.199346,0.759849,0.206950,3|0%,"POLYGON ((-86.57780 33.76532, -86.57780 33.766..."
3,4,8855.813477,4,1.0,None,None,717754,585183,49721,621226,56723,555217,465284,37403,488583,40112,717754.0,0.0,0.000000,0.815297,0.069273,0.865514,0.079028,0.773548,0.838022,0.067366,0.879986,0.072246,4|0%,"POLYGON ((-88.27462 33.53401, -88.27459 33.534..."
4,5,4067.180420,5,1.0,None,None,717754,495427,126226,543486,138438,560406,399106,97134,429649,102735,717754.0,0.0,0.000000,0.690246,0.175862,0.757204,0.192877,0.780777,0.712173,0.173328,0.766675,0.183322,5|0%,"MULTIPOLYGON (((-86.14981 34.53363, -86.14981 ..."
5,6,4556.976074,6,1.0,None,None,717755,517634,125785,552665,133963,552411,409699,93329,431902,96958,717754.0,1.0,0.000001,0.721185,0.175248,0.769991,0.186642,0.769637,0.741656,0.168948,0.781849,0.175518,6|0%,"POLYGON ((-87.03982 33.68746, -87.03982 33.687..."
6,7,10406.541992,7,1.0,None,None,717754,283337,378364,306688,387622,568540,239136,289398,255566,295119,717754.0,0.0,0.000000,0.394755,0.527150,0.427288,0.540049,0.792110,0.420614,0.509020,0.449513,0.519082,7|0%,"POLYGON ((-87.26692 33.51293, -87.26718 33.513..."


We need to confirm that there aren't any issues in the Districts, so we can use the "maup" package.

In [7]:
# Check geodataframe using maup's doctor function
maup.doctor(al_adopt_gdf)

/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/maup/repair.py:331: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlaps = inters[inters.area > 0].make_valid()


True

## Add the Congressional Districting Plans to the Voting Precinct Data

### Inital Plan:

In [8]:
# Import 2022 General Election voting data
al_data_gdf = gpd.read_file('al_gen_22_st_prec/al_gen_22_st_prec.shp')
al_data_gdf.columns

Index(['UNIQUE_ID', 'COUNTYFP', 'County', 'Precinct', 'G22A01NO', 'G22A01YES',
       'G22A02NO', 'G22A02YES', 'G22A03NO', 'G22A03YES', 'G22A04NO',
       'G22A04YES', 'G22A05NO', 'G22A05YES', 'G22A06NO', 'G22A06YES',
       'G22A07NO', 'G22A07YES', 'G22A08NO', 'G22A08YES', 'G22A09NO',
       'G22A09YES', 'G22A10NO', 'G22A10YES', 'G22AGRLCLA', 'G22AGROWRI',
       'G22AGRRPAT', 'G22AJ5DKEL', 'G22AJ5OWRI', 'G22AJ5RCOO', 'G22AJ6OWRI',
       'G22AJ6RWIS', 'G22ATGDMAJ', 'G22ATGOWRI', 'G22ATGRMAR', 'G22AUDLLAC',
       'G22AUDOWRI', 'G22AUDRSOR', 'G22CNSNO', 'G22CNSYES', 'G22GOVDFLO',
       'G22GOVLBLA', 'G22GOVOWRI', 'G22GOVRIVE', 'G22LTGLPAG', 'G22LTGOWRI',
       'G22LTGRAIN', 'G22PS1LBIS', 'G22PS1OWRI', 'G22PS1RODE', 'G22PS2LLAN',
       'G22PS2OWRI', 'G22PS2RBEE', 'G22SOSDLAF', 'G22SOSLSHE', 'G22SOSOWRI',
       'G22SOSRALL', 'G22TRELHAM', 'G22TREOWRI', 'G22TRERBOO', 'G22USSDBOY',
       'G22USSLSOP', 'G22USSOWRI', 'G22USSRBRI', 'geometry'],
      dtype='object')

In [9]:
# List of specific columns to keep
columns_to_keep = ['UNIQUE_ID', 'COUNTYFP', 'County', 'Precinct', 'geometry']

# Add columns starting with 'G22USS'
filtered_columns = columns_to_keep + [col for col in al_data_gdf.columns if col.startswith('G22USS')]

# Filter the GeoDataFrame
al_data_gdf = al_data_gdf[filtered_columns]

# Check that we've kept the right stuff
al_data_gdf

,UNIQUE_ID,COUNTYFP,County,Precinct,geometry,G22USSDBOY,G22USSLSOP,G22USSOWRI,G22USSRBRI
0,Jefferson-:-PREC 1010 - HUFFMAN BAPTIST CH,073,Jefferson,PREC 1010 - HUFFMAN BAPTIST CH,"POLYGON Z ((-86.64897 33.59396 0.00000, -86.64...",1864.0,39.0,1.0,447.0
1,Jefferson-:-PREC 1020 - TOM BRADFORD PARK,073,Jefferson,PREC 1020 - TOM BRADFORD PARK,"POLYGON Z ((-86.65986 33.64447 0.00000, -86.65...",1458.0,28.0,2.0,416.0
2,Jefferson-:-PREC 1030 - L_M_ SMITH MIDDLE,073,Jefferson,PREC 1030 - L_M_ SMITH MIDDLE,"POLYGON Z ((-86.68488 33.61943 0.00000, -86.68...",1372.0,25.0,1.0,164.0
3,Jefferson-:-PREC 1040 - BETHEL BAPTIST CHU,073,Jefferson,PREC 1040 - BETHEL BAPTIST CHU,"POLYGON Z ((-86.85019 33.55959 0.00000, -86.85...",1504.0,12.0,1.0,36.0
4,Jefferson-:-PREC 1060 - SUN VALLEY ELEMENT,073,Jefferson,PREC 1060 - SUN VALLEY ELEMENT,"POLYGON Z ((-86.70259 33.63731 0.00000, -86.70...",1472.0,12.0,4.0,206.0
...,...,...,...,...,...,...,...,...,...
1934,Choctaw-:-TOOMEY,023,Choctaw,TOOMEY,"POLYGON Z ((-88.35069 31.74690 0.00000, -88.35...",44.0,1.0,0.0,79.0
1935,Choctaw-:-TOXEY,023,Choctaw,TOXEY,"POLYGON Z ((-88.35634 31.95628 0.00000, -88.35...",80.0,1.0,0.0,150.0
1936,Choctaw-:-WOMACK HILL,023,Choctaw,WOMACK HILL,"POLYGON Z ((-88.22792 31.84026 0.00000, -88.22...",15.0,0.0,0.0,71.0
1937,Choctaw-:-YANTLEY,023,Choctaw,YANTLEY,"POLYGON Z ((-88.30748 32.21943 0.00000, -88.30...",154.0,2.0,0.0,19.0


In [10]:
# This is how we assign the precincts to congressional districts
precincts_to_districts_init_assignment = maup.assign(al_data_gdf.geometry, al_init_gdf.geometry)


/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/maup/intersections.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df = df[df.area > area_cutoff].reset_index(drop=True)
/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/maup/intersections.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/maup/assign.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).area)


### Adopted Plan

In [11]:
# How districts are inputted into the shapefile dataframe
precincts_to_districts_adopt_assignment = maup.assign(al_data_gdf.geometry, al_adopt_gdf.geometry)

/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/maup/intersections.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df = df[df.area > area_cutoff].reset_index(drop=True)
/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/maup/intersections.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/opt/anaconda3/envs/Math195/lib/python3.12/site-packages/maup/assign.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).area)


In [ ]:
# Were any blocks not assigned to a precint?
al_data_gdf[precincts_to_districts_adopt_assignment.isna()]
al_data_gdf[precincts_to_districts_init_assignment.isna()]

,UNIQUE_ID,COUNTYFP,County,Precinct,geometry,G22USSDBOY,G22USSLSOP,G22USSOWRI,G22USSRBRI


### Combining it all together

In [15]:
# Now, we need to assign each precinct their congressional district - we should be able to do this by adding a column
al_data_gdf["OriginalPlan"] = precincts_to_districts_init_assignment
al_data_gdf["AdoptedPlan"] = precincts_to_districts_adopt_assignment

al_data_gdf

,UNIQUE_ID,COUNTYFP,County,Precinct,geometry,G22USSDBOY,G22USSLSOP,G22USSOWRI,G22USSRBRI,OriginalPlan,AdoptedPlan
0,Jefferson-:-PREC 1010 - HUFFMAN BAPTIST CH,073,Jefferson,PREC 1010 - HUFFMAN BAPTIST CH,"POLYGON Z ((-86.64897 33.59396 0.00000, -86.64...",1864.0,39.0,1.0,447.0,5,6
1,Jefferson-:-PREC 1020 - TOM BRADFORD PARK,073,Jefferson,PREC 1020 - TOM BRADFORD PARK,"POLYGON Z ((-86.65986 33.64447 0.00000, -86.65...",1458.0,28.0,2.0,416.0,5,5
2,Jefferson-:-PREC 1030 - L_M_ SMITH MIDDLE,073,Jefferson,PREC 1030 - L_M_ SMITH MIDDLE,"POLYGON Z ((-86.68488 33.61943 0.00000, -86.68...",1372.0,25.0,1.0,164.0,5,6
3,Jefferson-:-PREC 1040 - BETHEL BAPTIST CHU,073,Jefferson,PREC 1040 - BETHEL BAPTIST CHU,"POLYGON Z ((-86.85019 33.55959 0.00000, -86.85...",1504.0,12.0,1.0,36.0,6,6
4,Jefferson-:-PREC 1060 - SUN VALLEY ELEMENT,073,Jefferson,PREC 1060 - SUN VALLEY ELEMENT,"POLYGON Z ((-86.70259 33.63731 0.00000, -86.70...",1472.0,12.0,4.0,206.0,5,6
...,...,...,...,...,...,...,...,...,...,...,...
1934,Choctaw-:-TOOMEY,023,Choctaw,TOOMEY,"POLYGON Z ((-88.35069 31.74690 0.00000, -88.35...",44.0,1.0,0.0,79.0,6,6
1935,Choctaw-:-TOXEY,023,Choctaw,TOXEY,"POLYGON Z ((-88.35634 31.95628 0.00000, -88.35...",80.0,1.0,0.0,150.0,6,6
1936,Choctaw-:-WOMACK HILL,023,Choctaw,WOMACK HILL,"POLYGON Z ((-88.22792 31.84026 0.00000, -88.22...",15.0,0.0,0.0,71.0,6,6
1937,Choctaw-:-YANTLEY,023,Choctaw,YANTLEY,"POLYGON Z ((-88.30748 32.21943 0.00000, -88.30...",154.0,2.0,0.0,19.0,6,6


## Join the Census Data with the Precinct Voting Data/Congressional Districts

In [ ]:
# Import the Census Data, which is in a .csv format
al_blocks = pd.read_csv('DECENNIALPL2020.ALABAMA/DECENNIALPL2020.P1-Data.csv')
print(al_blocks.columns)
#print(al_data_gdf.columns)


Index(['GEO_ID', 'NAME', 'P1_001N', 'P1_002N', 'P1_003N', 'P1_004N', 'P1_005N',
       'P1_006N', 'P1_007N', 'P1_008N', 'P1_009N', 'P1_010N', 'P1_011N',
       'P1_012N', 'P1_013N', 'P1_014N', 'P1_015N', 'P1_016N', 'P1_017N',
       'P1_018N', 'P1_019N', 'P1_020N', 'P1_021N', 'P1_022N', 'P1_023N',
       'P1_024N', 'P1_025N', 'P1_026N', 'P1_027N', 'P1_028N', 'P1_029N',
       'P1_030N', 'P1_031N', 'P1_032N', 'P1_033N', 'P1_034N', 'P1_035N',
       'P1_036N', 'P1_037N', 'P1_038N', 'P1_039N', 'P1_040N', 'P1_041N',
       'P1_042N', 'P1_043N', 'P1_044N', 'P1_045N', 'P1_046N', 'P1_047N',
       'P1_048N', 'P1_049N', 'P1_050N', 'P1_051N', 'P1_052N', 'P1_053N',
       'P1_054N', 'P1_055N', 'P1_056N', 'P1_057N', 'P1_058N', 'P1_059N',
       'P1_060N', 'P1_061N', 'P1_062N', 'P1_063N', 'P1_064N', 'P1_065N',
       'P1_066N', 'P1_067N', 'P1_068N', 'P1_069N', 'P1_070N', 'P1_071N',
       'Unnamed: 73'],
      dtype='object')


/var/folders/7_/ynn6zkss3sjb5nkrscffqq500000gn/T/ipykernel_18067/844121808.py:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  al_blocks = pd.read_csv('DECENNIALPL2020.ALABAMA/DECENNIALPL2020.P1-Data.csv')


In [18]:
# We will only analyze people that identify as one specific race, so we can get rid of the columns we don't need

# List of specific columns to keep
filtered_columns = ['GEO_ID', 'NAME', 'P1_001N', 'P1_002N', 'P1_003N', "P1_004N", "P1_005N", "P1_006N", "P1_007N", "P1_008N"]

# Filter the GeoDataFrame
al_blocks = al_blocks[filtered_columns]

# Check that we've kept the right stuff
al_blocks

,GEO_ID,NAME,P1_001N,P1_002N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N
0,Geography,Geographic Area Name,!!Total:,!!Total:!!Population of one race:,!!Total:!!Population of one race:!!White alone,!!Total:!!Population of one race:!!Black or A...,!!Total:!!Population of one race:!!American I...,!!Total:!!Population of one race:!!Asian alone,!!Total:!!Population of one race:!!Native Haw...,!!Total:!!Population of one race:!!Some Other...
1,1000000US010010201001000,"Block 1000, Block Group 1, Census Tract 201, A...",21,16,12,4,0,0,0,0
2,1000000US010010201001001,"Block 1001, Block Group 1, Census Tract 201, A...",34,32,18,11,0,2,0,1
3,1000000US010010201001002,"Block 1002, Block Group 1, Census Tract 201, A...",29,28,24,2,0,0,0,2
4,1000000US010010201001003,"Block 1003, Block Group 1, Census Tract 201, A...",17,16,16,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
185972,1000000US011339659003071,"Block 3071, Block Group 3, Census Tract 9659, ...",0,0,0,0,0,0,0,0
185973,1000000US011339659003072,"Block 3072, Block Group 3, Census Tract 9659, ...",0,0,0,0,0,0,0,0
185974,1000000US011339659003073,"Block 3073, Block Group 3, Census Tract 9659, ...",0,0,0,0,0,0,0,0
185975,1000000US011339659003074,"Block 3074, Block Group 3, Census Tract 9659, ...",11,10,10,0,0,0,0,0


In [19]:
# Import the shapefile block data
al_block_gdf = gpd.read_file("tl_2022_al_tabblock20/tl_2022_01_tabblock20.shp")

#al_block_gdf["GEOID20"]

#al_blocks["GEO_ID"]


In [20]:
# We now need to combine the shapefil block data with the Census data
# We can join on GEO_ID, but need to rename column of shapefile and cut down the geoid in csv
al_block_gdf['GEO_ID'] = al_block_gdf['GEOID20']
al_blocks["GEO_ID"] = al_blocks["GEO_ID"].str[9:]

In [ ]:
# Confirm that we have a matching column to join on! 
# This is just a test of one specific GEO_ID, but if it works most likely everything else worked
if ((al_block_gdf["GEO_ID"] == "010810410014051").any()):
    print("Found")
if ((al_blocks["GEO_ID"] == "010810410014051").any()):
    print("Found")
else:
    print("Not found")

Found
Found


In [23]:
# Now, we can actually do the join!
#al_block_gdf = al_block_gdf.merge(al_blocks, on = ["GEO_ID"], how = "left")
al_block_gdf
# Yay, it's joined!

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,HOUSING20,POP20,geometry,GEO_ID,NAME,P1_001N,P1_002N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N
0,01,081,041001,4051,010810410014051,Block 4051,G5040,R,None,None,S,0,44841,+32.6253951,-085.5868924,0,0,"POLYGON ((-85.58792 32.62678, -85.58786 32.626...",010810410014051,"Block 4051, Block Group 4, Census Tract 410.01...",0,0,0,0,0,0,0,0
1,01,081,041903,3022,010810419033022,Block 3022,G5040,R,None,None,S,1475062,4520,+32.5808332,-085.1232843,60,150,"POLYGON ((-85.14290 32.57395, -85.14267 32.574...",010810419033022,"Block 3022, Block Group 3, Census Tract 419.03...",150,142,92,48,2,0,0,0
2,01,079,979300,3016,010799793003016,Block 3016,G5040,R,None,None,S,492543,660,+34.5908982,-087.3612864,7,24,"POLYGON ((-87.36603 34.58670, -87.36601 34.586...",010799793003016,"Block 3016, Block Group 3, Census Tract 9793, ...",24,19,19,0,0,0,0,0
3,01,059,973400,1079,010599734001079,Block 1079,G5040,R,None,None,S,309100,121412,+34.4763029,-088.1485282,0,0,"POLYGON ((-88.15314 34.47997, -88.15308 34.480...",010599734001079,"Block 1079, Block Group 1, Census Tract 9734, ...",0,0,0,0,0,0,0,0
4,01,043,964700,2008,010439647002008,Block 2008,G5040,R,None,None,S,1328148,13745,+34.1420820,-086.8881645,17,45,"POLYGON ((-86.89510 34.14119, -86.89510 34.141...",010439647002008,"Block 2008, Block Group 2, Census Tract 9647, ...",45,45,42,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185971,01,051,030103,1066,010510301031066,Block 1066,G5040,R,None,None,S,1288156,0,+32.5875573,-086.3754563,1,2,"POLYGON ((-86.39136 32.60005, -86.39128 32.600...",010510301031066,"Block 1066, Block Group 1, Census Tract 301.03...",2,2,2,0,0,0,0,0
185972,01,073,010900,2004,010730109002004,Block 2004,G5040,U,07786,U,S,7003,0,+33.5872394,-086.7701725,0,0,"POLYGON ((-86.77091 33.58716, -86.77016 33.587...",010730109002004,"Block 2004, Block Group 2, Census Tract 109, J...",0,0,0,0,0,0,0,0
185973,01,101,005608,1022,011010056081022,Block 1022,G5040,R,None,None,S,393475,0,+32.3317240,-086.1143846,14,50,"POLYGON ((-86.12188 32.33084, -86.12188 32.330...",011010056081022,"Block 1022, Block Group 1, Census Tract 56.08,...",50,50,49,0,1,0,0,0
185974,01,125,010604,1027,011250106041027,Block 1027,G5040,R,None,None,S,35733,0,+33.2779824,-087.1522002,0,0,"POLYGON ((-87.15297 33.27954, -87.15141 33.278...",011250106041027,"Block 1027, Block Group 1, Census Tract 106.04...",0,0,0,0,0,0,0,0


In [36]:
# We now need to agregate the block data up to the precinct data
pop_cols = ['P1_001N', 'P1_002N', 'P1_003N', "P1_004N", "P1_005N", "P1_006N", "P1_007N", "P1_008N"]

# How blocks are inputted into the precinct shapefile dataframe
#blocks_to_precincts_assignment = maup.assign(al_block_gdf.geometry, al_data_gdf.geometry)

# We need to get all the data types of P1 to ints
for item in pop_cols:
    al_block_gdf[item] = pd.to_numeric(al_block_gdf[item])

al_data_gdf[pop_cols] = al_block_gdf[pop_cols].groupby(blocks_to_precincts_assignment).sum()

In [38]:
# Check that we didn't use any population
al_data_gdf[pop_cols].head()

print(al_block_gdf[pop_cols].sum())

print(al_data_gdf[pop_cols].sum())

P1_001N    5024279
P1_002N    4767326
P1_003N    3220452
P1_004N    1296162
P1_005N      33625
P1_006N      76660
P1_007N       2984
P1_008N     137443
dtype: int64
P1_001N    5024279
P1_002N    4767326
P1_003N    3220452
P1_004N    1296162
P1_005N      33625
P1_006N      76660
P1_007N       2984
P1_008N     137443
dtype: int64


In [41]:
al_data_gdf

,UNIQUE_ID,COUNTYFP,County,Precinct,geometry,G22USSDBOY,G22USSLSOP,G22USSOWRI,G22USSRBRI,OriginalPlan,AdoptedPlan,P1_001N,P1_002N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N
0,Jefferson-:-PREC 1010 - HUFFMAN BAPTIST CH,073,Jefferson,PREC 1010 - HUFFMAN BAPTIST CH,"POLYGON Z ((-86.64897 33.59396 0.00000, -86.64...",1864.0,39.0,1.0,447.0,5,6,10710,10363,2141,7632,49,56,0,485
1,Jefferson-:-PREC 1020 - TOM BRADFORD PARK,073,Jefferson,PREC 1020 - TOM BRADFORD PARK,"POLYGON Z ((-86.65986 33.64447 0.00000, -86.65...",1458.0,28.0,2.0,416.0,5,5,7929,7661,1829,5557,18,50,0,207
2,Jefferson-:-PREC 1030 - L_M_ SMITH MIDDLE,073,Jefferson,PREC 1030 - L_M_ SMITH MIDDLE,"POLYGON Z ((-86.68488 33.61943 0.00000, -86.68...",1372.0,25.0,1.0,164.0,5,6,6729,6506,698,5598,39,17,0,154
3,Jefferson-:-PREC 1040 - BETHEL BAPTIST CHU,073,Jefferson,PREC 1040 - BETHEL BAPTIST CHU,"POLYGON Z ((-86.85019 33.55959 0.00000, -86.85...",1504.0,12.0,1.0,36.0,6,6,4095,4038,78,3943,4,1,0,12
4,Jefferson-:-PREC 1060 - SUN VALLEY ELEMENT,073,Jefferson,PREC 1060 - SUN VALLEY ELEMENT,"POLYGON Z ((-86.70259 33.63731 0.00000, -86.70...",1472.0,12.0,4.0,206.0,5,6,5324,5188,673,4163,16,15,2,319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1934,Choctaw-:-TOOMEY,023,Choctaw,TOOMEY,"POLYGON Z ((-88.35069 31.74690 0.00000, -88.35...",44.0,1.0,0.0,79.0,6,6,287,279,167,110,0,1,0,1
1935,Choctaw-:-TOXEY,023,Choctaw,TOXEY,"POLYGON Z ((-88.35634 31.95628 0.00000, -88.35...",80.0,1.0,0.0,150.0,6,6,583,564,352,209,2,0,0,1
1936,Choctaw-:-WOMACK HILL,023,Choctaw,WOMACK HILL,"POLYGON Z ((-88.22792 31.84026 0.00000, -88.22...",15.0,0.0,0.0,71.0,6,6,138,135,108,26,0,0,0,1
1937,Choctaw-:-YANTLEY,023,Choctaw,YANTLEY,"POLYGON Z ((-88.30748 32.21943 0.00000, -88.30...",154.0,2.0,0.0,19.0,6,6,445,434,61,368,3,0,0,2


## Let's save the above work to a shapefile so we don't have to deal with it later...

In [40]:
al_data_gdf.to_file("Alabama_precincts_with_data.shp")

/var/folders/7_/ynn6zkss3sjb5nkrscffqq500000gn/T/ipykernel_18067/1594887714.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  al_data_gdf.to_file("Alabama_precincts_with_data.shp")
